In [1]:
                        ##-- BASIC EXPLATORY DATA ANALYSIS - 3 --##
# Importing libraries

import pandas as pd
import getpass
import math
import numpy as np
#from tqdm import trange
import gc
import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.ensemble import IsolationForest


if getpass.getuser() == "asus":
    file_path = "C:/Users/asus/Desktop/trendyol_data_v2/"
    
if getpass.getuser() == "your computers name":
    file_path = "C:/Users/..desiredlocation..."

My main aim is to gather all information (from all tables) into one dataframe to train my model. I am collecting information from tables one by one, considering the memory constraint.

In [2]:
main_data = pd.read_csv(file_path + 'merged_transactions_v7.csv',index_col=0) #as of Oct 12, 2021. Size of the file is 1.89 gb.

C:\Users\asus\AppData\Local\R-MINI~1\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
main_data.head()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_probability,return_probability_2
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000
3,3941815,0,163856,595758968,36888399,65613254,49.90,10.67,0,1,0.0,0,0,0,0.000000,1,0.213828,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.000000
4,15996105,1,968,595758987,96930601,161408176,71.99,9.40,0,0,0.0,0,0,1,0.200022,1,0.130574,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.083333


In [3]:
pd.set_option("display.max_columns", None)

In [4]:
main_data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772659 entries, 0 to 10772658
Data columns (total 45 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  Erkek                 int64  
 18  GenderlessItem        int64  
 19  Kadın                 int64  
 20  Kadın_Kız             int64  
 21  Unisex                int64  
 22  Altın                 int64  
 23  Bej  

We continue with the table 'supplier_defective_return'.

In [5]:
defective_return = pd.read_csv(file_path + 'supplier_defective_return_v2.csv')

In [6]:
defective_return.tail(5)

,supplier_id,returnRate
22531,110564,0.003115
22532,390815,0.003115
22533,121338,0.006231
22534,131868,0.012461
22535,233930,0.003009


In test data, there are only one predictor column, which is the concatenation of user id, product content id and order parent id.
We cannot use any other column that we store in the main dataframe as a predictor.

IDEA 1:
I am going to train a model with all the predictors except these three columns. In that way I can understand which factors do affect 
the return possibility. Considering the feature importances, I am going to appoint a coefficient to each of the predictors, and in my original
dataframe, the value of predictors will be multiplied by their associated coefficient, and these values will be added to 'is_returned' column.
Of course, new 'is_returned' column will not consist of binary values, instead the values will become floats.

IDEA 2:
user id  --> user demographics
product content id  --> product features
order parent id --> is_different_sizes column (will be derived within the test data)
un-concatenate the predictor column in test data and join these information stated above, train your model with training data, 
then fit the model to test data to do the prediction, then 
delete all columns in test data except user_id, product_content_id, order_parent_id and is_returned (prediction).
concatenate three id's. then you are done.\
But still I couldn't figure out how to exploit supplier_id.

I will go with idea 2.

In [7]:
len(main_data.supplier_id.unique())

22135

In [8]:
len(main_data.product_content_id.unique())

684112

In [13]:
defective_return.tail(3)

,supplier_id,suppliers_return_rate
22533,121338,0.006231
22534,131868,0.012461
22535,233930,0.003009


In [12]:
defective_return = defective_return.rename({'returnRate':'suppliers_return_rate'},axis=1)

In [14]:
main_data_v2 = pd.merge(main_data,defective_return, on = 'supplier_id', how ='left')

In [17]:
main_data_v2.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.003045
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0.000545
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000744


In [16]:
main_data_v2 = main_data_v2.rename({'return_probability':'return_prob_from_answers','return_probability_2':'return_prob_from_reviews'},axis=1)

In [18]:
del main_data
del defective_return

In [19]:
main_data_v2.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772659 entries, 0 to 10772658
Data columns (total 46 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   user_id                   int64  
 1   is_elite_user             int64  
 2   supplier_id               int64  
 3   order_parent_id           int64  
 4   product_content_id        int64  
 5   product_variant_id        int64  
 6   discounted_price          float64
 7   ship_cost                 float64
 8   is_wallet_trx             int64  
 9   is_saved_card_trx         int64  
 10  is_returned               float64
 11  coupon_used               int64  
 12  is_different_sizes        int64  
 13  is_discounted             int64  
 14  promotion_percent         float64
 15  is_shipcost_incurred      int64  
 16  shipping_percent          float64
 17  Erkek                     int64  
 18  GenderlessItem            int64  
 19  Kadın                     int64  
 20  Kadın_Kız             

In [20]:
# SUPPLIER DISPUTED RETURN
disputed_return = pd.read_csv(file_path + 'supplier_disputed_return_v2.csv')

In [21]:
disputed_return.head(3)

,supplier_id,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
0,344,164,0,0,0.0
1,263495,332,0,0,0.0
2,193158,670,1,0,0.0


In [22]:
main_data_v2.isna().sum()

user_id                      0
is_elite_user                0
supplier_id                  0
order_parent_id              0
product_content_id           0
product_variant_id           0
discounted_price             0
ship_cost                    0
is_wallet_trx                0
is_saved_card_trx            0
is_returned                  0
coupon_used                  0
is_different_sizes           0
is_discounted                0
promotion_percent            0
is_shipcost_incurred         0
shipping_percent             0
Erkek                        0
GenderlessItem               0
Kadın                        0
Kadın_Kız                    0
Unisex                       0
Altın                        0
Bej                          0
Beyaz                        0
Bordo                        0
Ekru                         0
Gri                          0
Gümüş                        0
Haki                         0
Kahverengi                   0
Kırmızı                      0
Lacivert

In [23]:
#na's in 'suppliers_return_rate' is due to the fact that some suppliers are not listed in defective_returns dataframe.
#Note: If a supplier is missing from this table, it means that the store for the supplier is closed.

In [24]:
# I simply choose to omit these na rows.
main_data_v2 = main_data_v2.dropna()

In [25]:
main_data_v2.isna().sum()

user_id                     0
is_elite_user               0
supplier_id                 0
order_parent_id             0
product_content_id          0
product_variant_id          0
discounted_price            0
ship_cost                   0
is_wallet_trx               0
is_saved_card_trx           0
is_returned                 0
coupon_used                 0
is_different_sizes          0
is_discounted               0
promotion_percent           0
is_shipcost_incurred        0
shipping_percent            0
Erkek                       0
GenderlessItem              0
Kadın                       0
Kadın_Kız                   0
Unisex                      0
Altın                       0
Bej                         0
Beyaz                       0
Bordo                       0
Ekru                        0
Gri                         0
Gümüş                       0
Haki                        0
Kahverengi                  0
Kırmızı                     0
Lacivert                    0
Mavi      

In [26]:
disputed_return.head(3)

,supplier_id,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
0,344,164,0,0,0.0
1,263495,332,0,0,0.0
2,193158,670,1,0,0.0


supplier_id: ID for the seller \
total_claim: Total number of returns for this seller.\
unresolved_claim: Total number of disputed claims for the seller.\
unresolved_accepted_claim: Total number of disputed claims that are decided on behalf of the buyer.\
unresolvedclaim_percentage: Ratio of unresolved_accepted_claim to unresolved_claim.

In [27]:
disputed_return.unresolved_accepted_claim.sum()

199486

In [35]:
main_data_v2.groupby(['supplier_id']).agg(
     total_sales_supplier = pd.NamedAgg(column='is_returned',aggfunc='count'))

,total_sales_supplier
supplier_id,
62,273
63,192233
67,178
73,321
85,156
...,...
430526,4
430613,2
430753,28


In [38]:
disputed_return.describe()

,supplier_id,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
count,20888.000000,2.088800e+04,20888.000000,20888.000000,20888.000000
mean,246467.258857,4.909012e+02,14.631942,9.550268,0.260746
std,96664.422995,2.297020e+04,111.495563,88.060700,0.380772
min,62.000000,1.000000e+00,0.000000,0.000000,0.000000
25%,167198.000000,6.000000e+00,0.000000,0.000000,0.000000
50%,240187.500000,2.100000e+01,0.000000,0.000000,0.000000
75%,328446.000000,9.100000e+01,4.000000,2.000000,0.531320
max,434994.000000,3.269293e+06,5673.000000,5102.000000,1.000000


In [39]:
#75% percentile for the last column is 0.53.
#Let's label those suppliers who have its last column more than 0.50 (see also below values)

In [42]:
disputed_return.unresolvedclaim_percentage.values

array([0.        , 0.        , 0.        , ..., 0.66666667, 0.8       ,
       0.92307692])

In [43]:
disputed_return['dispute_label'] = np.where(disputed_return.unresolvedclaim_percentage > 0.5, 1, 0)

In [45]:
disputed_return = disputed_return[['supplier_id','dispute_label']]

In [46]:
main_data_v3 = pd.merge(main_data_v2, disputed_return, on = 'supplier_id', how = 'left')

In [47]:
main_data_v3.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.003045,0.0
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0.000545,0.0
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000744,1.0


In [48]:
del main_data_v2 #to save memory

In [49]:
del disputed_return

In [50]:
# SUPPLIER RETURN
supplier_return = pd.read_csv(file_path + 'supplier_return_v2.csv')

In [51]:
supplier_return.head(3)

,supplier_id,returnRate
0,247019,0.0
1,264022,0.0
2,165838,0.0


In [52]:
supplier_return.describe()

,supplier_id,returnRate
count,22536.000000,22536.000000
mean,248854.356097,0.096032
std,96391.457295,0.091749
min,62.000000,0.000000
25%,170489.000000,0.027778
50%,243647.500000,0.073961
75%,330740.500000,0.141078
max,434994.000000,1.000000


In [54]:
supplier_return[supplier_return.returnRate > 0.14].returnRate.values

array([0.140625  , 0.140625  , 0.140625  , ..., 0.20772947, 0.1408308 ,
       0.21883289])

75% percentile corresponds to 14% in return rate, let's again label those suppliers who have more than 14% return rate. \
As one might notice my approaches are fairly simple. The reason is I still couldn't figure out how to use the information \
coming from the suppliers' side, also the deadline for the submissions is approaching.

In [55]:
supplier_return['return_label'] = np.where(supplier_return.returnRate > 0.14, 1, 0)

In [56]:
main_data_v4 = pd.merge(main_data_v3, supplier_return, on = 'supplier_id', how = 'left')

In [57]:
# I will try to join the dataframe named 'respondent_suppliers'. It was giving me memory error. 
# Now I am utilizing memory in a more efficient way, maybe I can join them now.

In [58]:
main_data_v4.head(3)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.003045,0.0,0.098178,0
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0.000545,0.0,0.050737,0
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000744,1.0,0.093355,0


In [59]:
del main_data_v3
del supplier_return

In [60]:
respondent_suppliers = pd.read_csv(file_path + 'respondent_suppliers.csv')

In [63]:
#main_data_v5 = pd.merge(main_data_v4, respondent_suppliers, on = 'supplier_id', how = 'left')

It failed again. Let's move on.

In [64]:
main_data_v4.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772622 entries, 0 to 10772621
Data columns (total 49 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   user_id                   int64  
 1   is_elite_user             int64  
 2   supplier_id               int64  
 3   order_parent_id           int64  
 4   product_content_id        int64  
 5   product_variant_id        int64  
 6   discounted_price          float64
 7   ship_cost                 float64
 8   is_wallet_trx             int64  
 9   is_saved_card_trx         int64  
 10  is_returned               float64
 11  coupon_used               int64  
 12  is_different_sizes        int64  
 13  is_discounted             int64  
 14  promotion_percent         float64
 15  is_shipcost_incurred      int64  
 16  shipping_percent          float64
 17  Erkek                     int64  
 18  GenderlessItem            int64  
 19  Kadın                     int64  
 20  Kadın_Kız             

In [65]:
#memory usage is 4 gb at the moment.
# Last table to join is 'user_demographics'.

In [67]:
user_demographics = pd.read_csv(file_path + 'user_demographics_v2.csv')

In [70]:
user_demographics #this data was heavily incomplete, that's why we only have two columns except the user_id.

,user_id,membership_date,gender
0,22013146,2019-05-09 11:11:17.413000+00:00,Bay
1,49707264,2020-12-19 18:32:16.240000+00:00,Bay
2,48146878,2020-11-28 12:26:46.870000+00:00,Bay
3,53528681,2021-02-01 13:16:24.117000+00:00,Bay
4,17523184,2018-09-30 22:07:07.623000+00:00,Bay
...,...,...,...
686665,16196014,2018-05-14 21:06:10.043000+00:00,Bayan
686666,16364821,2018-06-02 15:28:01.583000+00:00,Bayan
686667,14942279,2017-12-04 18:15:40.357000+00:00,Bayan
686668,20498924,2019-02-06 02:29:38.667000+00:00,Bayan


In [71]:
len(main_data_v4.user_id.unique())

706436

There are some users which are present in our main dataframe but not in user_demographics.

In [72]:
main_data_v4.head(2)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.003045,0.0,0.098178,0
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0.000545,0.0,0.050737,0


In [74]:
user_demographics.gender.describe() #most of the users are women.

count     686670
unique         2
top        Bayan
freq      562664
Name: gender, dtype: object

I believe gender is an important variable for prediction. 

In [75]:
user_demographics = user_demographics.drop(['membership_date'],axis=1)

In [76]:
main_data_v5 = pd.merge(main_data_v4,user_demographics, on = 'user_id', how = 'left')

In [77]:
main_data_v5.head(2)

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_prob_from_answers,return_prob_from_reviews,suppliers_return_rate,dispute_label,returnRate,return_label,gender
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.003045,0.0,0.098178,0,Bayan
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.047619,0.000545,0.0,0.050737,0,Bayan


In [79]:
del main_data_v4

In [83]:
main_data_v5.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10772622 entries, 0 to 10772621
Data columns (total 50 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   user_id                   int64  
 1   is_elite_user             int64  
 2   supplier_id               int64  
 3   order_parent_id           int64  
 4   product_content_id        int64  
 5   product_variant_id        int64  
 6   discounted_price          float64
 7   ship_cost                 float64
 8   is_wallet_trx             int64  
 9   is_saved_card_trx         int64  
 10  is_returned               float64
 11  coupon_used               int64  
 12  is_different_sizes        int64  
 13  is_discounted             int64  
 14  promotion_percent         float64
 15  is_shipcost_incurred      int64  
 16  shipping_percent          float64
 17  Erkek                     int64  
 18  GenderlessItem            int64  
 19  Kadın                     int64  
 20  Kadın_Kız             

In [87]:
#main_data_v5.isna().sum()
#main_data_v5 = main_data_v5.dropna()

In [88]:
main_data_v5.to_csv(file_path+'main_data_v5.csv')

In [89]:
#continued from explatory_da_v4.ipynb